In [1]:
##### Imports #####
from tqdm import tqdm
import numpy as np

In [2]:
##### Data #####
# Define input text (Hamlet excerpt, converted to lowercase)
data = """To be, or not to be, that is the question: Whether \
'tis nobler in the mind to suffer The slings and arrows of ou\
trageous fortune, Or to take arms against a sea of troubles A\
nd by opposing end them. To die—to sleep, No more; and by a s\
leep to say we end The heart-ache and the thousand natural sh\
ocks That flesh is heir to: 'tis a consummation Devoutly to b\
e wish'd. To die, to sleep; To sleep, perchance to dream—ay, \
there's the rub: For in that sleep of death what dreams may c\
ome, When we have shuffled off this mortal coil, Must give us\
 pause—there's the respect That makes calamity of so long lif\
e. For who would bear the whips and scorns of time, Th'oppres\
sor's wrong, the proud man's contumely, The pangs of dispriz'\
d love, the law's delay, The insolence of office, and the spu\
rns That patient merit of th'unworthy takes, When he himself \
might his quietus make""".lower()  # convert to lowercase for simplicity

# Create set of unique characters in the dataset
chars = set(data)

# Data size = number of characters in text
# Char size = number of unique characters (vocabulary size)
data_size, char_size = len(data), len(chars)
print(f'Data size: {data_size}, Char Size: {char_size}')

# Create mapping: character → index
char_to_idx = {c: i for i, c in enumerate(chars)}
# Create mapping: index → character
idx_to_char = {i: c for i, c in enumerate(chars)}

# Training data:
# train_X = all chars except last
# train_y = all chars except first (shifted by one position)
train_X, train_y = data[:-1], data[1:]


Data size: 866, Char Size: 32


In [3]:
##### Helper Functions #####

def oneHotEncode(text):
    # Create a zero vector of shape (char_size, 1)
    # char_size = number of unique characters in dataset
    output = np.zeros((char_size, 1))
    
    # Set the index corresponding to the given character = 1
    output[char_to_idx[text]] = 1
    
    return output


# Xavier Normalized Initialization for weight matrices
def initWeights(input_size, output_size):
    # Random uniform initialization in range [-1, 1]
    # Scaled by sqrt(6 / (input_size + output_size)) for Xavier normalization
    return np.random.uniform(-1, 1, (output_size, input_size)) * np.sqrt(6 / (input_size + output_size))


In [4]:
##### Activation Functions #####

def sigmoid(input, derivative=False):
    # Sigmoid activation: maps values to range (0, 1)
    # If derivative=True → return gradient (used in backprop)
    if derivative:
        return input * (1 - input)
    return 1 / (1 + np.exp(-input))


def tanh(input, derivative=False):
    # Tanh activation: maps values to range (-1, 1)
    # If derivative=True → return gradient (1 - input^2)
    if derivative:
        return 1 - input ** 2
    return np.tanh(input)


def softmax(input):
    # Softmax activation: converts raw scores into probabilities
    # Each value in output ∈ (0,1) and all sum to 1
    return np.exp(input) / np.sum(np.exp(input))


In [5]:
##### Long Short-Term Memory Network Class #####
class LSTM:
    def __init__(self, input_size, hidden_size, output_size, num_epochs, learning_rate):
        # Hyperparameters
        self.learning_rate = learning_rate
        self.hidden_size = hidden_size
        self.num_epochs = num_epochs

        # Forget Gate weights & bias
        self.wf = initWeights(input_size, hidden_size)
        self.bf = np.zeros((hidden_size, 1))

        # Input Gate weights & bias
        self.wi = initWeights(input_size, hidden_size)
        self.bi = np.zeros((hidden_size, 1))

        # Candidate Gate weights & bias
        self.wc = initWeights(input_size, hidden_size)
        self.bc = np.zeros((hidden_size, 1))

        # Output Gate weights & bias
        self.wo = initWeights(input_size, hidden_size)
        self.bo = np.zeros((hidden_size, 1))

        # Final output layer weights & bias
        self.wy = initWeights(hidden_size, output_size)
        self.by = np.zeros((output_size, 1))

    # Reset network memory (hidden, cell states, and gates)
    def reset(self):
        self.concat_inputs = {}

        self.hidden_states = {-1: np.zeros((self.hidden_size, 1))}
        self.cell_states = {-1: np.zeros((self.hidden_size, 1))}

        self.activation_outputs = {}
        self.candidate_gates = {}
        self.output_gates = {}
        self.forget_gates = {}
        self.input_gates = {}
        self.outputs = {}

    # Forward Propagation
    def forward(self, inputs):
        self.reset()   # reset memory before each forward pass
        outputs = []

        for q in range(len(inputs)):
            # Concatenate hidden state and input vector
            self.concat_inputs[q] = np.concatenate((self.hidden_states[q - 1], inputs[q]))

            # Compute gate activations
            self.forget_gates[q]   = sigmoid(np.dot(self.wf, self.concat_inputs[q]) + self.bf)
            self.input_gates[q]    = sigmoid(np.dot(self.wi, self.concat_inputs[q]) + self.bi)
            self.candidate_gates[q]= tanh(np.dot(self.wc, self.concat_inputs[q]) + self.bc)
            self.output_gates[q]   = sigmoid(np.dot(self.wo, self.concat_inputs[q]) + self.bo)

            # Update cell state
            self.cell_states[q] = self.forget_gates[q] * self.cell_states[q - 1] + \
                                  self.input_gates[q] * self.candidate_gates[q]

            # Update hidden state
            self.hidden_states[q] = self.output_gates[q] * tanh(self.cell_states[q])

            # Compute output (before softmax)
            outputs += [np.dot(self.wy, self.hidden_states[q]) + self.by]

        return outputs

    # Backward Propagation Through Time
    def backward(self, errors, inputs):
        # Initialize gradients for all weights and biases
        d_wf, d_bf = 0, 0
        d_wi, d_bi = 0, 0
        d_wc, d_bc = 0, 0
        d_wo, d_bo = 0, 0
        d_wy, d_by = 0, 0

        # Initialize next hidden/cell state errors
        dh_next, dc_next = np.zeros_like(self.hidden_states[0]), np.zeros_like(self.cell_states[0])

        # Iterate backwards through sequence
        for q in reversed(range(len(inputs))):
            error = errors[q]

            # Final output layer error
            d_wy += np.dot(error, self.hidden_states[q].T)
            d_by += error

            # Error for hidden state
            d_hs = np.dot(self.wy.T, error) + dh_next

            # Output gate gradient
            d_o = tanh(self.cell_states[q]) * d_hs * sigmoid(self.output_gates[q], derivative=True)
            d_wo += np.dot(d_o, inputs[q].T)
            d_bo += d_o

            # Cell state gradient
            d_cs = tanh(tanh(self.cell_states[q]), derivative=True) * self.output_gates[q] * d_hs + dc_next

            # Forget gate gradient
            d_f = d_cs * self.cell_states[q - 1] * sigmoid(self.forget_gates[q], derivative=True)
            d_wf += np.dot(d_f, inputs[q].T)
            d_bf += d_f

            # Input gate gradient
            d_i = d_cs * self.candidate_gates[q] * sigmoid(self.input_gates[q], derivative=True)
            d_wi += np.dot(d_i, inputs[q].T)
            d_bi += d_i
            
            # Candidate gate gradient
            d_c = d_cs * self.input_gates[q] * tanh(self.candidate_gates[q], derivative=True)
            d_wc += np.dot(d_c, inputs[q].T)
            d_bc += d_c

            # Concatenated input error (sum of errors from all gates)
            d_z = np.dot(self.wf.T, d_f) + np.dot(self.wi.T, d_i) + np.dot(self.wc.T, d_c) + np.dot(self.wo.T, d_o)

            # Propagate error to next timestep
            dh_next = d_z[:self.hidden_size, :]
            dc_next = self.forget_gates[q] * d_cs

        # Clip gradients to avoid exploding gradients
        for d_ in (d_wf, d_bf, d_wi, d_bi, d_wc, d_bc, d_wo, d_bo, d_wy, d_by):
            np.clip(d_, -1, 1, out=d_)

        # Gradient descent parameter updates
        self.wf += d_wf * self.learning_rate
        self.bf += d_bf * self.learning_rate

        self.wi += d_wi * self.learning_rate
        self.bi += d_bi * self.learning_rate

        self.wc += d_wc * self.learning_rate
        self.bc += d_bc * self.learning_rate

        self.wo += d_wo * self.learning_rate
        self.bo += d_bo * self.learning_rate

        self.wy += d_wy * self.learning_rate
        self.by += d_by * self.learning_rate

    # Training function
    def train(self, inputs, labels):
        inputs = [oneHotEncode(input) for input in inputs]

        for _ in tqdm(range(self.num_epochs)):
            predictions = self.forward(inputs)

            errors = []
            for q in range(len(predictions)):
                # Cross-entropy error signal
                errors += [-softmax(predictions[q])]
                errors[-1][char_to_idx[labels[q]]] += 1

            # Backpropagation through time
            self.backward(errors, self.concat_inputs)
    
    # Testing function
    def test(self, inputs, labels):
        accuracy = 0
        probabilities = self.forward([oneHotEncode(input) for input in inputs])

        output = ''
        for q in range(len(labels)):
            # Sample prediction from softmax probabilities
            prediction = idx_to_char[np.random.choice(
                [*range(char_size)], p=softmax(probabilities[q].reshape(-1))
            )]

            output += prediction

            if prediction == labels[q]:
                accuracy += 1

        print(f'Ground Truth:\n\t{labels}\n')
        print(f'Predictions:\n\t{"".join(output)}\n')
        print(f'Accuracy: {round(accuracy * 100 / len(inputs), 2)}%')


In [6]:
# Initialize Network
hidden_size = 25

lstm = LSTM(input_size = char_size + hidden_size, hidden_size = hidden_size, output_size = char_size, num_epochs = 1_000, learning_rate = 0.05)

##### Training #####
lstm.train(train_X, train_y)

##### Testing #####
lstm.test(train_X, train_y)

100%|██████████| 1000/1000 [00:47<00:00, 20.97it/s]

Ground Truth:
	o be, or not to be, that is the question: whether 'tis nobler in the mind to suffer the slings and arrows of outrageous fortune, or to take arms against a sea of troubles and by opposing end them. to die—to sleep, no more; and by a sleep to say we end the heart-ache and the thousand natural shocks that flesh is heir to: 'tis a consummation devoutly to be wish'd. to die, to sleep; to sleep, perchance to dream—ay, there's the rub: for in that sleep of death what dreams may come, when we have shuffled off this mortal coil, must give us pause—there's the respect that makes calamity of so long life. for who would bear the whips and scorns of time, th'oppressor's wrong, the proud man's contumely, the pangs of dispriz'd love, the law's delay, the insolence of office, and the spurns that patient merit of th'unworthy takes, when he himself might his quietus make

Predictions:
	o be, or not to be, that is the question: whether 'tis nobler in the mind to suffer the slings and arrow